# Functional Connectivity
1. [x] Create a BIDS object
2. [x] Read the Atlas
3. [x] Create BIDSDataGrabber Node
4. [x] Create a FC Node which does the following for each subject - 
    5. [x] Extract ROI time series.
    6. [x] Create FC Maps [ROI x Voxels]
    7. [x] Group it in one of two groups: Autistic and TD

In [1]:
from bids.grabbids import BIDSLayout
from nipype.interfaces.fsl import (BET, ExtractROI, FAST, FLIRT, ImageMaths,
                                   MCFLIRT, SliceTimer, Threshold,Info)
from nipype.interfaces.afni import Resample
from nipype.interfaces.io import DataSink
from nipype.pipeline import Node, MapNode, Workflow, JoinNode
from nipype.interfaces.utility import IdentityInterface, Function
import os
from os.path import join as opj
from nipype.interfaces import afni
import nibabel as nib
import numpy as np

Failed to import duecredit due to No module named 'duecredit.version'


In [2]:
# Set paths

path_cwd = os.getcwd()
path_split_list = path_cwd.split('/')
s = path_split_list[0:-1]
s = opj('/',*s) # *s converts list to path, # also it is very important to add '/' in the begining so it is read as directory later
base_directory = opj(s,'result')
data_directory = opj(s,"data/ABIDE-BIDS-Preprocessed/NYU/")

mask_file = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')

In [3]:
layout = BIDSLayout(data_directory)

In [4]:
layout.get_subjects() # list the subject IDs of all the subjects

['0050952',
 '0050953',
 '0050954',
 '0050955',
 '0050956',
 '0050957',
 '0050958',
 '0050959',
 '0050960',
 '0050961',
 '0050962',
 '0050964',
 '0050965',
 '0050966',
 '0050967',
 '0050968',
 '0050969',
 '0050970',
 '0050971',
 '0050972',
 '0050973',
 '0050974',
 '0050975',
 '0050976',
 '0050977',
 '0050978',
 '0050979',
 '0050980',
 '0050981',
 '0050982',
 '0050983',
 '0050984',
 '0050985',
 '0050986',
 '0050987',
 '0050988',
 '0050989',
 '0050990',
 '0050991',
 '0050992',
 '0050993',
 '0050994',
 '0050995',
 '0050996',
 '0050997',
 '0050998',
 '0050999',
 '0051000',
 '0051001',
 '0051002',
 '0051003',
 '0051006',
 '0051007',
 '0051008',
 '0051009',
 '0051010',
 '0051011',
 '0051012',
 '0051013',
 '0051014',
 '0051015',
 '0051016',
 '0051017',
 '0051018',
 '0051019',
 '0051020',
 '0051021',
 '0051023',
 '0051024',
 '0051025',
 '0051026',
 '0051027',
 '0051028',
 '0051029',
 '0051030',
 '0051032',
 '0051033',
 '0051034',
 '0051035',
 '0051036',
 '0051038',
 '0051039',
 '0051040',
 '00

In [5]:
number_of_subjects = len(layout.get_subjects()) # Number of subjects you wish to preprocess


In [159]:
subject_id = layout.get_subjects()[0] # gives the first subject's ID

# Below command gies a list of pathnames
func_file_path = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
# anat_file_path = [f.filename for f in layout.get(subject=subject_id, type='T1w', extensions=['nii', 'nii.gz'])]


metadata = layout.get_metadata(path=func_file_path[0])

In [160]:
layout.get(subject=subject_id) # I used this to see the 'type' arg for anatomical and functional files 

[File(filename='/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz', subject='0050952', type='T1w', modality='anat'),
 File(filename='/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz', subject='0050952', run='1', type='bold', task='rest', modality='func')]

In [161]:
func_file_path# , anat_file_path

['/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz']

In [162]:
# imoportant specs about functional data

TR  = metadata['RepetitionTime']

num_volumes = int(metadata['NumberofMeasurements']) - 4 

metadata; # just add or remove the semicolon to supress or see the output



In [163]:
# code to find how many voxels are in the brain region using the mask

num_brain_voxels = 0
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

x_dim, y_dim, z_dim = mask_data.shape

for i in range(x_dim):
    for j in range(y_dim):
        for k in range(z_dim):
            if mask[i,j,k] == 1:
                num_brain_voxels = num_brain_voxels + 1


In [164]:
num_brain_voxels

228483

In [165]:
metadata;
# The metadata doesnot contain the true specs of the brain files as the brain files are now preprocessed.
# Only some fields are consistent now. For example SpacingBetweenSlices, SliceTiming are NOT valid anymore.
# RepetitionTime is still valid.
# NumberofMeasurements is invalid as 4 initial volumes are skipped
# So, use carefully.

## Atlas

In [11]:
# Put in the path of atlas you wish to use
atlasPath = opj(s,'atlas/Full_brain_atlas_thr0-2mm/fullbrain_atlas_thr0-2mm.nii.gz')

In [12]:
# Read the atlas
atlasObject = nib.load(atlasPath)
atlas = atlasObject.get_data()

### Lets see the number of regions in the atlas and display the atlas.

In [13]:
num_ROIs = int((np.max(atlas) - np.min(atlas) ))

print('Min Index:', np.min(atlas),'Max Index', np.max(atlas))
print('Total Number of Parcellations = ',num_ROIs)

Min Index: 0.0 Max Index 274.0
Total Number of Parcellations =  274


Index 0.0 denotes the regin of the scan which does not contain brain voxel.

Let's display the atlas.

In [169]:
# %matplotlib inline
# from nilearn import plotting
# plotting.plot_roi(atlasObject)


## Read the data files using BIDSGrabber

In [170]:
# List of Subject IDs

subject_list = (layout.get_subjects())[0:number_of_subjects]

Create our own custom function - BIDSDataGrabber using a Function Interface.

In [171]:
def get_nifti_filenames(subject_id,data_dir):
#     Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    anat_file_path = [f.filename for f in layout.get(subject=subject_id, type='T1w', extensions=['nii', 'nii.gz'])]
    func_file_path = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    
    return anat_file_path[0],func_file_path[0]



Wrap it inside a Node

In [172]:
BIDSDataGrabber = Node(Function(function=get_nifti_filenames, input_names=['subject_id','data_dir'],
                                output_names=['anat_file_path','func_file_path']), name='BIDSDataGrabber')
BIDSDataGrabber.iterables = [('subject_id',subject_list)]
BIDSDataGrabber.inputs.data_dir = data_directory

In [173]:
To test the function wrapped in the node

BIDSDataGrabber.inputs.data_dir = data_directory
BIDSDataGrabber.inputs.subject_id = layout.get_subjects()[0] # gives the first subject's ID
res = BIDSDataGrabber.run()

res.outputs

## Create a FC node

This node:
1. Exracts the average time series of the brain ROI's using the atlas and stores 
    it as a matrix of size [ROIs x Volumes].
2. Extracts the Voxel time series and stores it in matrix of size [Voxels x Volumes]


TODO: Check how nifti masker works. How does it summarize an ROI? Does it take the mean of all the voxels?

In [174]:
# Initialize a matrix of ROI time series and voxel time series

ROI_matrix = np.zeros((num_ROIs, num_volumes))
voxel_matrix = np.zeros((num_brain_voxels, num_volumes))

In [ ]:
# I tested the following to check if it extract the subject ID from the input file path
func_file_path[0].split('/')[-1].split('.')[0].split('_')[0].split('-')[1]

In [ ]:
# Fill up the voxel_matrix 

brain_data = nib.load(func_file_path[0])
brain = brain_data.get_data()

# %time gives incorrect time. check!
x_dim, y_dim, z_dim, _ = brain.shape

voxel_counter = 0
for i in range(x_dim):
    for j in range(y_dim):
        for k in range(z_dim):
            if mask[i,j,k] == 1:
                voxel_matrix[voxel_counter,:] = brain[i,j,k,:] 
                voxel_counter = voxel_counter + 1




In [ ]:
%%time
# Fill up the ROI_matrix

# Keep track of number of voxels per ROI as well by using an array - num_voxels_in_ROI[]


num_voxels_in_ROI = np.zeros((num_ROIs,1)) # A column arrray containing number of voxels in each ROI

for i in range(x_dim):
    for j in range(y_dim):
        for k in range(z_dim):
            label = int(atlas[i,j,k]) - 1
            if label != -1:
                ROI_matrix[label,:] = np.add(ROI_matrix[label,:], brain[i,j,k,:])
                num_voxels_in_ROI[label,0] = num_voxels_in_ROI[label,0] + 1
                
ROI_matrix = np.divide(ROI_matrix,num_voxels_in_ROI) # Check if divide is working correctly


In [ ]:
# Calculate the Correlation matrix [ROI x Voxels]
# Check if numpy's correlation function expects the mean centered data or not
# -> Couldn't find out. So make your own function to calculate pearson correlation 

In [ ]:
X, Y = ROI_matrix, voxel_matrix
# Subtract mean from X and Y

X = np.subtract(X, np.mean(X))
Y = np.subtract(Y, np.mean(Y))

temp1 = np.dot(X,Y.T)
print "Temp1 Done"
temp2 = np.sqrt(np.multiply(X,X))
print "Temp2 Done"
temp3 = np.sqrt(np.multiply(Y,Y)).T
print "Temp3 Done"
temp4 = np.dot(temp2,temp3)
print "Temp4 Done"
pear = np.divide(temp1, (temp4+0.0001))
print "Pear Done"

In [176]:
# pear.shape

Looks gr8!

1. Now, Lets encapsulate the above code inside a function and then in a node that will take as input the func file path and creates the FC map for that.

2. Next Node will be a join node that will create a two groups

3. Can we do the 1step by using some other node type like map?

In [ ]:
# Create a the function

def pear_coff(in_file, atlas, num_brain_voxels, mask):
    
    # imports
    import numpy as np
    import nibabel as nib
    import os
    from os.path import join as opj
    
#     # Put in the path of atlas you wish to use
#     atlasPath = opj(s,'atlas/Full_brain_atlas_thr0-2mm/fullbrain_atlas_thr0-2mm.nii.gz')
    
#     # Read the atlas
#     atlasObject = nib.load(atlasPath)
#     atlas = atlasObject.get_data()
    os.chdir('/home/jovyan/work/preprocess/temp/')
    
    
    num_ROIs = int((np.max(atlas) - np.min(atlas) ))

#     print('Min Index:', np.min(atlas),'Max Index', np.max(atlas))
#     print('Total Number of Parcellations = ',num_ROIs)
    
    
    
    # Read the brain in_file

    brain_data = nib.load(in_file)
    brain = brain_data.get_data()

    # %time gives incorrect time. check!
    x_dim, y_dim, z_dim, num_volumes = brain.shape

    
    # Initialize a matrix of ROI time series and voxel time series

    ROI_matrix = np.zeros((num_ROIs, num_volumes))
    voxel_matrix = np.zeros((num_brain_voxels, num_volumes))
    
    # Fill up the voxel_matrix 
#     print 'Filling up the voxel matrix'
    voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    voxel_matrix[voxel_counter,:] = brain[i,j,k,:] 
                    voxel_counter = voxel_counter + 1
    
    # Fill up the ROI_matrix
    
#     print 'FIlling up the ROI matrix'
    # Keep track of number of voxels per ROI as well by using an array - num_voxels_in_ROI[]

    num_voxels_in_ROI = np.zeros((num_ROIs,1)) # A column arrray containing number of voxels in each ROI

    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                label = int(atlas[i,j,k]) - 1
                if label != -1:
                    ROI_matrix[label,:] = np.add(ROI_matrix[label,:], brain[i,j,k,:])
                    num_voxels_in_ROI[label,0] = num_voxels_in_ROI[label,0] + 1

    ROI_matrix = np.divide(ROI_matrix,num_voxels_in_ROI) # Check if divide is working correctly
    
#     print 'Done'
    # Calculate the Correlation matrix [ROI x Voxels]
    X, Y = ROI_matrix, voxel_matrix
    
    # Subtract mean from X and Y
    X = np.subtract(X, np.mean(X))
    Y = np.subtract(Y, np.mean(Y))

    temp1 = np.dot(X,Y.T)
    temp2 = np.sqrt(np.multiply(X,X))
    temp3 = np.sqrt(np.multiply(Y,Y)).T
    temp4 = np.dot(temp2,temp3)
    pear_matrix = np.divide(temp1, (temp4+0.0001))
#     print "Pear Done"
    
    sub_id = in_file.split('/')[-1].split('.')[0].split('_')[0].split('-')[1]
    
    fc_file_name = sub_id + 'fc_map'
    
    np.save(fc_file_name, pear_matrix)
    
    pear_matrix_file = opj(os.getcwd(),fc_file_name) # path
    return pear_matrix_file
#     pear_matrix = 1
#     return pear_matrix
    
    

In [ ]:
import os
from os.path import join as opj
fc_file_name ='varun'
fc_file_path = opj(os.getcwd(),fc_file_name)
fc_file_path

In [ ]:
# Create the Node and set default valus to

pearcoff = Node(Function(function=pear_coff, input_names=['in_file','atlas','num_brain_voxels','mask'],
                                output_names=['pear_matrix']), name='pearcoff')

pearcoff.inputs.in_file = func_file_path[0]
pearcoff.inputs.atlas = atlas
pearcoff.inputs.num_brain_voxels = num_brain_voxels
pearcoff.inputs.mask = mask

In [ ]:
%%time
pearcoff.run()

In [ ]:
# Okay Lets try one more time:
# Create a the function

def pear_coff_2(in_file, atlas_file, mask_file):
    # code to find how many voxels are in the brain region using the mask
    
        # imports
    import numpy as np
    import nibabel as nib
    import os
    from os.path import join as opj

    
    os.chdir('/home/jovyan/work/preprocess/temp/')
    
    num_brain_voxels = 0
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()

    x_dim, y_dim, z_dim = mask_data.shape

    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    num_brain_voxels = num_brain_voxels + 1
                    
    atlasPath = atlas_file
    # Read the atlas
    atlasObject = nib.load(atlasPath)
    atlas = atlasObject.get_data()
    
    num_ROIs = int((np.max(atlas) - np.min(atlas) ))

#     print('Min Index:', np.min(atlas),'Max Index', np.max(atlas))
#     print('Total Number of Parcellations = ',num_ROIs)

    # Read the brain in_file

    brain_data = nib.load(in_file)
    brain = brain_data.get_data()

    x_dim, y_dim, z_dim, num_volumes = brain.shape
    
    # Initialize a matrix of ROI time series and voxel time series

    ROI_matrix = np.zeros((num_ROIs, num_volumes))
    voxel_matrix = np.zeros((num_brain_voxels, num_volumes))
    
    # Fill up the voxel_matrix 
    print('Filling up voxel matrix')
    voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    voxel_matrix[voxel_counter,:] = brain[i,j,k,:] 
                    voxel_counter = voxel_counter + 1

                    
    # Fill up the ROI_matrix
    print('Filling up voxel matrix')
    # Keep track of number of voxels per ROI as well by using an array - num_voxels_in_ROI[]

    num_voxels_in_ROI = np.zeros((num_ROIs,1)) # A column arrray containing number of voxels in each ROI

    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                label = int(atlas[i,j,k]) - 1
                if label != -1:
                    ROI_matrix[label,:] = np.add(ROI_matrix[label,:], brain[i,j,k,:])
                    num_voxels_in_ROI[label,0] = num_voxels_in_ROI[label,0] + 1

    ROI_matrix = np.divide(ROI_matrix,num_voxels_in_ROI) # Check if divide is working correctly

    X, Y = ROI_matrix, voxel_matrix
    # Subtract mean from X and Y

    X = np.subtract(X, np.mean(X))
    Y = np.subtract(Y, np.mean(Y))

    temp1 = np.dot(X,Y.T)
    print ("Temp1 Done")
    temp2 = np.sqrt(np.multiply(X,X))
    print ("Temp2 Done")
    temp3 = np.sqrt(np.multiply(Y,Y)).T
    print ("Temp3 Done")
    temp4 = np.dot(temp2,temp3)
    print ("Temp4 Done")
    coff_matrix = np.divide(temp1, (temp4+0.0001))
    print ("Pear Done")
#     print ('Over!')
#     coff_matrix = 1
    sub_id = in_file.split('/')[-1].split('.')[0].split('_')[0].split('-')[1]
    
    fc_file_name = sub_id + 'fc_map'
    
    np.save(fc_file_name, coff_matrix)
    
    coff_matrix_file = opj(os.getcwd(),fc_file_name) # path
    return coff_matrix_file



In [ ]:
# Again Create the Node and set default valus to

pearcoff = Node(Function(function=pear_coff_2, input_names=['in_file','atlas_file','mask_file'],
                                output_names=['coff_matrix_file']), name='pearcoff')

pearcoff.inputs.in_file = func_file_path[0]
pearcoff.inputs.atlas_file = atlasPath
# pearcoff.inputs.num_brain_voxels = num_brain_voxels
pearcoff.inputs.mask_file = mask_file

In [ ]:
%%time
pearcoff.run()

Above are the two ways the node can be created:
> You can pass the whole matrix as input (or)  
> You can pass just the file name and let the function compute the matrix

The comparison between two :  The second approach is faster: 11.2 sec vs 15.1 sec. Next to confirm, run the second function first then the second function and then check.

**Takeaway:** The system hangs if you are trying to return big matrices. Solution is to save the matrix and return the path of it.



## Create a JOIN Node

Create a Join Node that takes all the FC maps created and group them in two groups by looking at the subject ID [Later try to read the participants.tsv and use the data given in it to see to which group the subject belongs].  
Output: Two 3D tensors, each of size [ROI x Voxels x Num_subjects]

NO!

It would be very difficult to keep all that data in memory.

Instead of creating tensors, group filenames in two lists - Autistic and TD. And return these two lists.
Hmmm.. Looks good :)

----------------------------------------------------------------------------------------------------------

Then for each group, loop over all the entries of matrix - [i,j,:] But again storing such a big matrix is not possible:

SOLUTION:  

Mem-map the stored array, and then access the second row directly from disk:

> X = np.load('/tmp/123.npy', mmap_mode='r')  
> X[1, :]  

memmap([4, 5, 6]) # Second row of the array.

Source: https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.load.html

Solution is to loop over the Matrix and select only one row. This will be of size 500/174 MB  
Do this for all the files. 

In the first pass:  

* First row of all the subjects will be read. Size = 500(for each FC map)/174(number of ROIs)  * ~174(subjects) 
Therefore the size of data in first pass will be ~500 MB.
* After reading the data, store it into a temporary array [remember to overwrite this in next pass]
* The size of the temporary array will be [Num_subjects x voxels] - This denote the corr values 1st ROI with of all voxels.
* Perform T-test on each of the columns.

NEXT: Think about how to store the results of T test and apply FDR and how to map it back to brain.[when the above works fine]







### Lets load the FC matrix in chunks

In [ ]:
X = np.load('/home/jovyan/work/preprocess/temp/0050952fc_map.npy', mmap_mode='r')

In [ ]:
corr_chunk = X[0,:]

In [ ]:
corr_chunk1 = X[1,:]

In [ ]:
X.shape, corr_chunk.shape, corr_chunk1.shape

2.06 GB occupied before reading 
2.09 GB after reading

one row with 228483 values occupied 0.03 GB = 0.03*1024 = 30.72 MB

30.72 MB is occupied for one row of one subject

30.72 * 184 = 5648.8 MB ~ 5.5 GB will be occupied for one ROI's analysis

   ### Create the node again with the workspace

Func Connectivity function with minor changes so that the FC matrix is saved in the Node's folder

In [177]:
def pear_coff_2(in_file, atlas_file, mask_file, num_brain_voxels):
    # code to find how many voxels are in the brain region using the mask
    
        # imports
    import numpy as np
    import nibabel as nib
    import os
    from os.path import join as opj

    
#     os.chdir('/home/jovyan/work/preprocess/temp/')
    
#     num_brain_voxels = 0
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()

    x_dim, y_dim, z_dim = mask_data.shape

#     for i in range(x_dim):
#         for j in range(y_dim):
#             for k in range(z_dim):
#                 if mask[i,j,k] == 1:
#                     num_brain_voxels = num_brain_voxels + 1
                    
    atlasPath = atlas_file
    # Read the atlas
    atlasObject = nib.load(atlasPath)
    atlas = atlasObject.get_data()
    
    num_ROIs = int((np.max(atlas) - np.min(atlas) ))

#     print('Min Index:', np.min(atlas),'Max Index', np.max(atlas))
#     print('Total Number of Parcellations = ',num_ROIs)

    # Read the brain in_file

    brain_data = nib.load(in_file)
    brain = brain_data.get_data()

    x_dim, y_dim, z_dim, num_volumes = brain.shape
    
    # Initialize a matrix of ROI time series and voxel time series

    ROI_matrix = np.zeros((num_ROIs, num_volumes))
    voxel_matrix = np.zeros((num_brain_voxels, num_volumes))
    
    # Fill up the voxel_matrix 
#     print('Filling up voxel matrix')
    voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    voxel_matrix[voxel_counter,:] = brain[i,j,k,:] 
                    voxel_counter = voxel_counter + 1

                    
    # Fill up the ROI_matrix
#     print('Filling up voxel matrix')
    # Keep track of number of voxels per ROI as well by using an array - num_voxels_in_ROI[]

    num_voxels_in_ROI = np.zeros((num_ROIs,1)) # A column arrray containing number of voxels in each ROI

    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                label = int(atlas[i,j,k]) - 1
                if label != -1:
                    ROI_matrix[label,:] = np.add(ROI_matrix[label,:], brain[i,j,k,:])
                    num_voxels_in_ROI[label,0] = num_voxels_in_ROI[label,0] + 1

    ROI_matrix = np.divide(ROI_matrix,num_voxels_in_ROI) # Check if divide is working correctly

    X, Y = ROI_matrix, voxel_matrix
    # Subtract mean from X and Y

    X = np.subtract(X, np.mean(X))
    Y = np.subtract(Y, np.mean(Y))

    temp1 = np.dot(X,Y.T)
#     print ("Temp1 Done")
    temp2 = np.sqrt(np.multiply(X,X))
#     print ("Temp2 Done")
    temp3 = np.sqrt(np.multiply(Y,Y)).T
#     print ("Temp3 Done")
    temp4 = np.dot(temp2,temp3)
#     print ("Temp4 Done")
    coff_matrix = np.divide(temp1, (temp4+0.0001))
#     print ("Pear Done")
#     print ('Over!')
#     coff_matrix = 1
    sub_id = in_file.split('/')[-1].split('.')[0].split('_')[0].split('-')[1]
    
    fc_file_name = sub_id + '_fc_map'
    
    np.save(fc_file_name, coff_matrix)
    
    
    fc_file_name = fc_file_name + '.npy'
    
    coff_matrix_file = opj(os.getcwd(),fc_file_name) # path
    return coff_matrix_file



In [178]:
# Again Create the Node and set default values to paths

pearcoff = Node(Function(function=pear_coff_2, input_names=['in_file','atlas_file','mask_file','num_brain_voxels'],
                                output_names=['coff_matrix_file']), name='pearcoff')

# pearcoff.inputs.in_file = func_file_path[0]
pearcoff.inputs.atlas_file = atlasPath
pearcoff.inputs.num_brain_voxels = num_brain_voxels
pearcoff.inputs.mask_file = mask_file

In [179]:
# %%time
# pearcoff.run()

## Datasink
I needed to define the structure of what files are saved and where.

In [180]:
# Create DataSink object
dataSink = Node(DataSink(), name='datasink')

# Name of the output folder
dataSink.inputs.base_directory = opj(base_directory,'datasink_fc')




To create the substitutions I looked the `datasink` folder where I was redirecting the output. I manually selected the part of file/folder name that I wanted to change and copied below to be substituted.  

**TODO:** Using datasink create a hierarchical directory structure i.e. folder in folder - to exactly match BIDS.

In [181]:
# Define substitution strings so that the data is similar to BIDS
substitutions = [('_subject_id_', 'sub-')]

# Feed the substitution strings to the DataSink node
dataSink.inputs.substitutions = substitutions

# ('_resample_brain_flirt.nii_brain', ''),
# ('_roi_st_mcf_flirt.nii_brain_flirt', ''),
                

### Create a Join node to gather the FC maps computed 

First lets create the function to do hypothesis testing:  
* It reads ROI i (Starting from 1) row of all the subjects in the FC matrix and saves it in a matrix of size [num_subjects x voxels]
* Loops over the elements of each column, converts to z-score and then does a t-test
* Each column gives the variation of corr coff of ROI i with the voxel corresponding to the column.
* After this operation you get a vector of p-values of size- number of voxels telling how significant the voxel connectivity is with the ROI i 

In [182]:
def hypothesis_test_temp(fc_file_list):
    print('######################## File List ######################: ',fc_file_list)
#     file_handle = []
    
#     # store all the fc file handles in a list
#     for file_path in fc_file_list:
#         file_handle.append(np.load(file_path, mmap_mode='r'))
        
     
#     # extracting the shape of one FC map and checking the num of rows and columns
#     number_of_ROIs, number_of_voxels = file_handle[0].shape 
    
#     # define the matrix for storing the voxels corr values corresponding to ith ROI for all subjects
#     all_subject_corr_matrix = np.zeros((len(file_handle), number_of_voxels))
    
    
#     # reads the rows one by one of the fc map for all subjects in Autism group
#     for roi in range(number_of_ROIs):
#         for participant_index in range(len(file_handle)):
#             all_subject_corr_matrix[participant_index,:] = file_handle[participant_index][roi,:]
    
#         # do z-standardization of corr values individually for each column and then do a t-test individually for each column
#         # use : https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.zscore.html
#         # and then : https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ttest_ind.html
            
            
    
# #     print('Number of file handles: ',len(file_handle))
# #     print('FC File list ',fc_file_list)
    significance_matrix = 1
    return significance_matrix
# Dummy function that I am not using anywhere (I forgot why I created this)    

### Following function does the Hypothesis testing and calculates the p-values and stores them in a big matrix of size [num_ROIs x num_brain_voxels] . File is called significance_matrix.

In [183]:
def hypothesis_test(fc_file_list):
    
    # imports
    from scipy import stats
    import numpy as np
    import os
    from os.path import join as opj

    
    
    file_handle = []
    
    # store all the fc file handles in a list
    for file_path in fc_file_list:
        file_handle.append(np.load(file_path, mmap_mode='r'))
        
     
    # extracting the shape of one FC map and checking the num of rows and columns
    number_of_ROIs, number_of_voxels = file_handle[0].shape 
    
    # define the matrix for storing the voxels corr values corresponding to ith ROI for all subjects
    all_subject_corr_matrix = np.zeros((len(file_handle), number_of_voxels))
    
    # Matrix to store the p-values which results from doing t-test
    
    significance_matrix = np.zeros((file_handle[0].shape))
    
#     number_of_ROIs = 1
    # reads the rows one by one of the fc map for all subjects in Autism group
    for roi in range(number_of_ROIs):
        for participant_index in range(len(file_handle)):
            all_subject_corr_matrix[participant_index,:] = file_handle[participant_index][roi,:]
    
        # do z-standardization of corr values individually for each column and then do a t-test individually for each column
        # use : https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.zscore.html
        for voxel in range(number_of_voxels):
            all_subject_corr_matrix[:,voxel] = stats.mstats.zscore(all_subject_corr_matrix[:,voxel])        
        # and then : https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ttest_ind.html
        # Divide into two groups - 0-78 - Autistic , 79-183 -TD 
            
            autism_start_index = 0
            autism_end_index = 1 #78
            td_start_index = 2 #79
            td_end_index = 3 #183
        
            _,significance_matrix[roi,voxel] = stats.ttest_ind(all_subject_corr_matrix[autism_start_index:autism_end_index+1,voxel],
                                                                   all_subject_corr_matrix[td_start_index:td_end_index+1,voxel])
    
    print('Now going to save significance matrix')
    np.save('significance_matrix',significance_matrix)
    
    significance_matrix_file_name ='significance_matrix.npy'
    
    
    significance_matrix_file = opj(os.getcwd(),significance_matrix_file_name) # path
    
    return significance_matrix_file


In [184]:
# def hypothesis_test1(fc_file_list):
#     print('FC File list ',fc_file_list)
#     significance_matrix = 1
#     return significance_matrix
    
    
#     Not able to print???? Check why???? 
# I was tring to print the names of files. It got printed first then It stopped printing. Thats why the above question.

In [185]:
hypothesisTest = JoinNode(Function(function=hypothesis_test, input_names=['fc_file_list'],
                 output_names=['significance_matrix_file']),
                 joinsource="BIDSDataGrabber",
                 joinfield="fc_file_list",
                 name="hypothesisTest")

### Create the Workflow

In [186]:
wf = Workflow(name="functionalConnectivity")
base_dir = opj(s,'result')
wf.base_dir = base_dir # Dir where all the outputs will be stored(inside BETFlow folder).

wf.connect([      (BIDSDataGrabber, pearcoff, [('func_file_path','in_file')]),
                  (pearcoff,dataSink,[('coff_matrix_file','fc_maps.@coff_matrix_file')]),
                  (pearcoff,hypothesisTest,[('coff_matrix_file','fc_file_list')])
           ])
# Run it in parallel
%time wf.run('MultiProc', plugin_args={'n_procs': 8})

171024-22:31:55,248 workflow INFO:
	 Workflow functionalConnectivity settings: ['check', 'execution', 'logging']
171024-22:31:55,632 workflow INFO:
	 Running in parallel.
171024-22:31:55,641 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a183 ID: 0
171024-22:31:55,642 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a183 jobid: 0
171024-22:31:55,643 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a182 ID: 3
171024-22:31:55,644 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a182 jobid: 3
171024-22:31:55,645 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a181 ID: 6
171024-22:31:55,647 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a181 jobid: 6
171024-22:31:55,648 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a180 ID: 9
171024-22:31:55,650 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a180 jobid: 9
171024-22:31:55,651 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a179 ID: 12
171024-22:31:55,653 workflow INFO:
	 [Job finished] jobname: B

171024-22:31:55,810 workflow INFO:
	 Executing node datasink.a175 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051149/datasink
171024-22:31:55,813 workflow INFO:
	 Executing: datasink.a173 ID: 32
171024-22:31:55,814 workflow INFO:
	 Executing node datasink.a174 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051148/datasink
171024-22:31:55,816 workflow INFO:
	 Executing: datasink.a172 ID: 35
171024-22:31:55,817 workflow INFO:
	 Executing node datasink.a173 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051147/datasink
171024-22:31:55,819 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a171 ID: 36
171024-22:31:55,819 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051149/0051149_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051149/0051149_fc_map.npy
171024-22:31:55,821 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a171 jobid: 36
171024-22:31:55,824 workflow INFO:
	 Executing: B

	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051124/0051124_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051124/0051124_fc_map.npy

171024-22:31:55,956 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a162 jobid: 63
171024-22:31:55,959 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a161 ID: 66
171024-22:31:55,961 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a161 jobid: 66
171024-22:31:55,962 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a160 ID: 69
171024-22:31:55,963 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a160 jobid: 69
171024-22:31:55,965 workflow INFO:
	 [Job finished] jobname: datasink.a167 jobid: 50
171024-22:31:55,966 workflow INFO:
	 [Job finished] jobname: datasink.a166 jobid: 53
171024-22:31:55,968 workflow INFO:
	 [Job finished] jobname: datasink.a165 jobid: 56
171024-22:31:55,969 workflow INFO:
	 [Job finished] jobname: datasink.a164 jobid: 59
171024-22:31:55,974 workflow INFO:
	 Executing: pearco

171024-22:31:56,110 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a151 jobid: 96
171024-22:31:56,111 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a150 ID: 99
171024-22:31:56,112 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a150 jobid: 99
171024-22:31:56,113 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a149 ID: 102
171024-22:31:56,114 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a149 jobid: 102
171024-22:31:56,116 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a148 ID: 105
171024-22:31:56,117 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a148 jobid: 105
171024-22:31:56,122 workflow INFO:
	 Executing: datasink.a155 ID: 86
171024-22:31:56,125 workflow INFO:
	 Executing: datasink.a154 ID: 89
171024-22:31:56,126 workflow INFO:
	 Executing node datasink.a155 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051115/datasink
171024-22:31:56,128 workflow INFO:
	 Executing: datasink.a153 ID: 92
171024-22:31:56,129 workf

171024-22:31:56,369 workflow INFO:
	 [Job finished] jobname: datasink.a137 jobid: 140
171024-22:31:56,371 workflow INFO:
	 [Job finished] jobname: datasink.a136 jobid: 143
171024-22:31:56,377 workflow INFO:
	 Executing: datasink.a135 ID: 146
171024-22:31:56,380 workflow INFO:
	 Executing: datasink.a134 ID: 149
171024-22:31:56,381 workflow INFO:
	 Executing node datasink.a135 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051095/datasink
171024-22:31:56,383 workflow INFO:
	 Executing: datasink.a133 ID: 152
171024-22:31:56,383 workflow INFO:
	 Executing node datasink.a134 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051094/datasink
171024-22:31:56,386 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051095/0051095_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051095/0051095_fc_map.npy
171024-22:31:56,386 workflow INFO:
	 Executing: datasink.a132 ID: 155
171024-22:31:56,391 workflow INFO:
	 Executing node data

...

171024-22:31:57,425 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a060 ID: 369
171024-22:31:57,425 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a060 jobid: 369
171024-22:31:57,430 workflow INFO:
	 Executing: datasink.a067 ID: 350
171024-22:31:57,432 workflow INFO:
	 Executing: datasink.a066 ID: 353
171024-22:31:57,433 workflow INFO:
	 Executing node datasink.a067 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051023/datasink
171024-22:31:57,435 workflow INFO:
	 Executing: datasink.a065 ID: 356
171024-22:31:57,436 workflow INFO:
	 Executing node datasink.a066 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0051021/datasink
171024-22:31:57,438 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051023/0051023_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051023/0051023_fc_map.npy
171024-22:31:57,441 workflow INFO:
	 Executing: datasink.a064 ID: 359
171024-22:31:57,442 workflow INFO:
	 Executing node 

	 Executing node datasink.a046 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0050999/datasink
171024-22:31:57,698 workflow INFO:
	 Executing node datasink.a045 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0050998/datasink171024-22:31:57,697 workflow INFO:
	 Executing: pearcoff.a044 ID: 418
171024-22:31:57,695 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0051000/0051000_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0051000/0051000_fc_map.npy
171024-22:31:57,698 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0050999/0050999_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0050999/0050999_fc_map.npy

171024-22:31:57,701 workflow INFO:
	 [Job finished] jobname: pearcoff.a044 jobid: 418
171024-22:31:57,702 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a043 ID: 420
171024-22:31:57,704 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a043 jobid: 420
171024-22:31:

171024-22:31:57,816 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a032 ID: 453
171024-22:31:57,817 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a032 jobid: 453
171024-22:31:57,819 workflow INFO:
	 [Job finished] jobname: datasink.a039 jobid: 434
171024-22:31:57,821 workflow INFO:
	 [Job finished] jobname: datasink.a038 jobid: 437
171024-22:31:57,822 workflow INFO:
	 [Job finished] jobname: datasink.a037 jobid: 440
171024-22:31:57,827 workflow INFO:
	 Executing: datasink.a036 ID: 443
171024-22:31:57,830 workflow INFO:
	 Executing: pearcoff.a035 ID: 445
171024-22:31:57,831 workflow INFO:
	 Executing node datasink.a036 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0050989/datasink
171024-22:31:57,832 workflow INFO:
	 [Job finished] jobname: pearcoff.a035 jobid: 445
171024-22:31:57,834 workflow INFO:
	 Executing: pearcoff.a034 ID: 448
171024-22:31:57,835 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0050989/0050989_fc_map.npy 

171024-22:31:57,959 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a023 jobid: 480
171024-22:31:57,960 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a022 ID: 483
171024-22:31:57,961 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a022 jobid: 483
171024-22:31:57,962 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a021 ID: 486
171024-22:31:57,963 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a021 jobid: 486
171024-22:31:57,965 workflow INFO:
	 [Job finished] jobname: datasink.a028 jobid: 467
171024-22:31:57,969 workflow INFO:
	 Executing: datasink.a027 ID: 470
171024-22:31:57,972 workflow INFO:
	 Executing: datasink.a026 ID: 473
171024-22:31:57,973 workflow INFO:
	 Executing node datasink.a027 in dir: /home1/varunk/result/functionalConnectivity/_subject_id_0050980/datasink
171024-22:31:57,975 workflow INFO:
	 Executing: datasink.a025 ID: 476
171024-22:31:57,976 workflow INFO:
	 Executing node datasink.a026 in dir: /home1/varunk/result/functionalConn

171024-22:31:58,287 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a003 jobid: 540
171024-22:31:58,290 workflow INFO:
	 Executing: BIDSDataGrabber.aI.a002 ID: 543
171024-22:31:58,295 workflow INFO:
	 [Job finished] jobname: BIDSDataGrabber.aI.a002 jobid: 543
171024-22:31:58,314 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0050961/0050961_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0050961/0050961_fc_map.npy
171024-22:31:58,317 interface INFO:
	 sub: /home1/varunk/result/datasink_fc/fc_maps/_subject_id_0050960/0050960_fc_map.npy -> /home1/varunk/result/datasink_fc/fc_maps/sub-0050960/0050960_fc_map.npy
171024-22:31:58,323 workflow INFO:
	 [Job finished] jobname: datasink.a009 jobid: 524
171024-22:31:58,326 workflow INFO:
	 [Job finished] jobname: datasink.a008 jobid: 527
171024-22:31:58,330 workflow INFO:
	 Executing: datasink.a005 ID: 536
171024-22:31:58,334 workflow INFO:
	 Executing: pearcoff.a004 ID: 538
171024-22:31:58,335 

In [187]:
# Visualize the detailed graph
from IPython.display import Image
wf.write_graph(graph2use='exec', format='png', simple_form=True)
file_name = opj(base_directory,'functionalConnectivity/graph_detailed.dot.png')
Image(filename=file_name)

171025-02:43:29,454 workflow INFO:
	 Generated workflow graph: /home1/varunk/result/functionalConnectivity/graph.dot.png (graph2use=exec, simple_form=True).


## Now use the significance matrix file and do FDR Correction on the list of FC map got for each subject


In [89]:
# Gathering the FC (functional connectivity) file names
# Next: Edit the join node to save the list in a file.

fc_file_list = ['/home1/varunk/result/functionalConnectivity/_subject_id_0050952/pearcoff/0050952_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050953/pearcoff/0050953_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050954/pearcoff/0050954_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050955/pearcoff/0050955_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050956/pearcoff/0050956_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050957/pearcoff/0050957_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050958/pearcoff/0050958_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050959/pearcoff/0050959_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050960/pearcoff/0050960_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050961/pearcoff/0050961_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050962/pearcoff/0050962_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050964/pearcoff/0050964_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050965/pearcoff/0050965_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050966/pearcoff/0050966_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050967/pearcoff/0050967_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050968/pearcoff/0050968_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050969/pearcoff/0050969_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050970/pearcoff/0050970_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050971/pearcoff/0050971_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050972/pearcoff/0050972_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050973/pearcoff/0050973_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050974/pearcoff/0050974_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050975/pearcoff/0050975_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050976/pearcoff/0050976_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050977/pearcoff/0050977_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050978/pearcoff/0050978_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050979/pearcoff/0050979_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050980/pearcoff/0050980_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050981/pearcoff/0050981_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050982/pearcoff/0050982_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050983/pearcoff/0050983_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050984/pearcoff/0050984_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050985/pearcoff/0050985_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050986/pearcoff/0050986_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050987/pearcoff/0050987_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050988/pearcoff/0050988_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050989/pearcoff/0050989_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050990/pearcoff/0050990_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050991/pearcoff/0050991_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050992/pearcoff/0050992_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050993/pearcoff/0050993_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050994/pearcoff/0050994_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050995/pearcoff/0050995_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050996/pearcoff/0050996_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050997/pearcoff/0050997_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050998/pearcoff/0050998_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0050999/pearcoff/0050999_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051000/pearcoff/0051000_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051001/pearcoff/0051001_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051002/pearcoff/0051002_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051003/pearcoff/0051003_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051006/pearcoff/0051006_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051007/pearcoff/0051007_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051008/pearcoff/0051008_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051009/pearcoff/0051009_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051010/pearcoff/0051010_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051011/pearcoff/0051011_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051012/pearcoff/0051012_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051013/pearcoff/0051013_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051014/pearcoff/0051014_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051015/pearcoff/0051015_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051016/pearcoff/0051016_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051017/pearcoff/0051017_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051018/pearcoff/0051018_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051019/pearcoff/0051019_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051020/pearcoff/0051020_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051021/pearcoff/0051021_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051023/pearcoff/0051023_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051024/pearcoff/0051024_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051025/pearcoff/0051025_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051026/pearcoff/0051026_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051027/pearcoff/0051027_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051028/pearcoff/0051028_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051029/pearcoff/0051029_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051030/pearcoff/0051030_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051032/pearcoff/0051032_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051033/pearcoff/0051033_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051034/pearcoff/0051034_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051035/pearcoff/0051035_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051036/pearcoff/0051036_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051038/pearcoff/0051038_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051039/pearcoff/0051039_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051040/pearcoff/0051040_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051041/pearcoff/0051041_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051042/pearcoff/0051042_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051044/pearcoff/0051044_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051045/pearcoff/0051045_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051046/pearcoff/0051046_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051047/pearcoff/0051047_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051048/pearcoff/0051048_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051049/pearcoff/0051049_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051050/pearcoff/0051050_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051051/pearcoff/0051051_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051052/pearcoff/0051052_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051053/pearcoff/0051053_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051054/pearcoff/0051054_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051055/pearcoff/0051055_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051056/pearcoff/0051056_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051057/pearcoff/0051057_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051058/pearcoff/0051058_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051059/pearcoff/0051059_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051060/pearcoff/0051060_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051061/pearcoff/0051061_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051062/pearcoff/0051062_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051063/pearcoff/0051063_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051064/pearcoff/0051064_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051065/pearcoff/0051065_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051066/pearcoff/0051066_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051067/pearcoff/0051067_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051068/pearcoff/0051068_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051069/pearcoff/0051069_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051070/pearcoff/0051070_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051071/pearcoff/0051071_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051072/pearcoff/0051072_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051073/pearcoff/0051073_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051074/pearcoff/0051074_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051075/pearcoff/0051075_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051076/pearcoff/0051076_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051077/pearcoff/0051077_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051078/pearcoff/0051078_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051079/pearcoff/0051079_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051080/pearcoff/0051080_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051081/pearcoff/0051081_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051082/pearcoff/0051082_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051083/pearcoff/0051083_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051084/pearcoff/0051084_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051085/pearcoff/0051085_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051086/pearcoff/0051086_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051087/pearcoff/0051087_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051088/pearcoff/0051088_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051089/pearcoff/0051089_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051090/pearcoff/0051090_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051091/pearcoff/0051091_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051093/pearcoff/0051093_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051094/pearcoff/0051094_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051095/pearcoff/0051095_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051096/pearcoff/0051096_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051097/pearcoff/0051097_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051098/pearcoff/0051098_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051099/pearcoff/0051099_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051100/pearcoff/0051100_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051101/pearcoff/0051101_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051102/pearcoff/0051102_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051103/pearcoff/0051103_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051104/pearcoff/0051104_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051105/pearcoff/0051105_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051106/pearcoff/0051106_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051107/pearcoff/0051107_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051108/pearcoff/0051108_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051109/pearcoff/0051109_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051110/pearcoff/0051110_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051111/pearcoff/0051111_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051112/pearcoff/0051112_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051113/pearcoff/0051113_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051114/pearcoff/0051114_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051115/pearcoff/0051115_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051116/pearcoff/0051116_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051117/pearcoff/0051117_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051118/pearcoff/0051118_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051119/pearcoff/0051119_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051120/pearcoff/0051120_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051121/pearcoff/0051121_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051122/pearcoff/0051122_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051123/pearcoff/0051123_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051124/pearcoff/0051124_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051125/pearcoff/0051125_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051126/pearcoff/0051126_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051127/pearcoff/0051127_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051128/pearcoff/0051128_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051129/pearcoff/0051129_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051130/pearcoff/0051130_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051131/pearcoff/0051131_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051146/pearcoff/0051146_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051147/pearcoff/0051147_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051148/pearcoff/0051148_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051149/pearcoff/0051149_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051150/pearcoff/0051150_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051151/pearcoff/0051151_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051152/pearcoff/0051152_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051153/pearcoff/0051153_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051154/pearcoff/0051154_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051155/pearcoff/0051155_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051156/pearcoff/0051156_fc_map.npy', '/home1/varunk/result/functionalConnectivity/_subject_id_0051159/pearcoff/0051159_fc_map.npy']

In [90]:
len(fc_file_list)

184

## Parallelizing the significance_marrix file creation
* Following code computes the p-values by distributing each ROI's correlation with all voxels to each processor

In [91]:
%%time

# Write the program to comepute the significance matrix parallely: 

# def hypothesis_test_parallel(fc_file_list):
    
# imports
from scipy import stats
import numpy as np
import os
from os.path import join as opj
from multiprocessing import Pool
import multiprocessing.managers



class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)



significance_matrix_file_name ='significance_matrix.npy'
significance_matrix_file = opj(os.getcwd(),significance_matrix_file_name) # path
print('File name: ',significance_matrix_file)


#     file_handle = []

#     # store all the fc file handles in a list
#     for file_path in fc_file_list:
#         file_handle.append(np.load(file_path, mmap_mode='r'))


# extracting the shape of one FC map and checking the num of rows and columns
#     number_of_ROIs, number_of_voxels = file_handle[0].shape 
number_of_ROIs, number_of_voxels = np.load(fc_file_list[0], mmap_mode='r').shape


# define the matrix for storing the voxels corr values corresponding to ith ROI for all subjects
all_subject_corr_matrix = np.zeros((len(fc_file_list), number_of_voxels))

# Matrix to store the p-values which results from doing t-test

# significance_matrix = np.zeros((number_of_ROIs, number_of_voxels))



#     significance_matrix = np.memmap('significance_matrix', dtype='float32', mode='w+', shape=(file_handle[0].shape))

# number_of_ROIs = 8
# reads the rows one by one of the fc map for all subjects in Autism group


def calc_significance(roi):

    file_handle = []   # Again coz file handle cannot be pickled

    # store all the fc file handles in a list
    for file_path in fc_file_list:                             
        file_handle.append(np.load(file_path, mmap_mode='r'))


#     global significance_matrix
    for participant_index in range(len(file_handle)):
        all_subject_corr_matrix[participant_index,:] = stats.mstats.zscore(file_handle[participant_index][roi,:])

    # do z-standardization of corr values individually for each column and then do a t-test individually for each column
    # use : https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.zscore.html

    
#     Error:
#     for voxel in range(number_of_voxels):
#         all_subject_corr_matrix[:,voxel] = stats.mstats.zscore(all_subject_corr_matrix[:,voxel])        

#    Correct:
#         all_subject_corr_matrix[participant_index,:] = stats.mstats.zscore(file_handle[participant_index][roi,:])
#          Instead edit the above uncommented line by adding stats.mstats.zscore() around it 
    
    
    # and then : https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.ttest_ind.html
    # Divide into two groups - 0-78 - Autistic , 79-183 -TD 

    for voxel in range(number_of_voxels):   
        autism_start_index = 0
        autism_end_index = 78
        td_start_index = 79
        td_end_index = 157 #183- total

        _,significance_matrix[roi,voxel] = stats.ttest_ind(all_subject_corr_matrix[autism_start_index:autism_end_index+1,voxel],
                                                               all_subject_corr_matrix[td_start_index:td_end_index+1,voxel])

    return 1

m = MyManager()
m.start()
significance_matrix = m.np_zeros((number_of_ROIs, number_of_voxels))

pool = Pool(8)

pool_inputs = np.arange(number_of_ROIs)
data_outputs = pool.map(calc_significance, pool_inputs)


print('Now going to save significance matrix')
np.save('significance_matrix_22',significance_matrix)

#     del significance_matrix # Deletion flushes memory changes to disk before removing the object

significance_matrix_file_name ='significance_matrix.npy'

print('Saved!')

significance_matrix_file = opj(os.getcwd(),significance_matrix_file_name) # path

# return significance_matrix_file


File name:  /home1/varunk/result/p_value_files/significance_matrix.npy
Now going to save significance matrix
Saved!
CPU times: user 17.3 s, sys: 17.8 s, total: 35.1 s
Wall time: 1h 40min 29s


# ERROR:
In the above calc_significance function, I have done the z-standardaziation of each column, whereas I was supposed to do it for each row as all the voxels of an individual brain must be standardardized, not the same voxel for each subject that I has done. (Changed it above). 

In [15]:
# In the parallel version of calc_significance, 
# each instance got its own seperate copy of the significant 
# You can use a custom Manager to create a shared numpy array
# https://stackoverflow.com/questions/25938187/trying-to-use-multiprocessing-to-fill-an-array-in-python


import multiprocessing.managers



class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)

m = MyManager()
m.start()
significance_matrix = m.np_zeros((number_of_ROIs, number_of_voxels))

pool = Pool(8)
# following statement disappeared: check it again
func = partial(calc_significance, significance_matrix)

pool_inputs = np.arange(number_of_ROIs)
data_outputs = pool.map(calc_significance, pool_inputs)

### FDR using FSl (partially done)
* I first thaught that I will use FDR correction of FSL
* It took as input a p-value brain file
* Following code reads the big p-value matrix and extract p-values of voxels corresponding to each ROI
* Creates a separate brain file for it and saves it using nibabel.
* BUT I did't do the FDR correction coz I used the python's inbuilt FDS correction function

In [72]:
# Next I have to create the p-value file for each ROI - This will result in 274 brain files
# import mask : mask_file
# output: mask file converted to p value-file

def p_value_brain_creater(brain_voxel_list):
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()
    
    # to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
    brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
    brain_data = nib.load(brain_file)
    brain = brain_data.get_data()

    
    
    
    roi_number = brain_voxel_list[0]
    brain_voxel_list = brain_voxel_list[1:]
    
    x_dim, y_dim, z_dim =mask.shape

    brain_voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                    brain_voxel_counter = brain_voxel_counter + 1
                    
    
    
    
#     np.max(mask)
    
    
    if brain_voxel_counter == len(brain_voxel_list):
        print("All p-values successfully fitted ROI:",roi_number)
        filename = 'roi_'+ str(int(roi_number)) + '_p_value_file.nii.gz'
        
        
        brain_with_header = nib.Nifti1Image(brain, affine=brain_data.affine,header = brain_data.header)
        nib.save(brain_with_header,filename)
        
        
     

        print('Saved!')

        p_value_file = opj(os.getcwd(),filename) # path of file
        
        return p_value_file
    else:
        print("p-values fitting failed")
        return 'N.A'


In [ ]:
# read the significance matrix
X = np.load('/home1/varunk/Autism-Connectome-Analysis-bids-related/significance_matrix.npy', mmap_mode='r')
# stack a column indentifying the roi value
X = np.c_[1:X.shape[0]+1,X]  # appended the ROI number to the ROI-voxels correlation vector

# THIS WAS DONE SO THAT THE NODE KNOWS FOR WHICH ROI NUMBER THE p-value BRAIN FILE IS BEING CREATED 



In [59]:
# test the above code if it creates p-value brain images:

brain_voxel_list = X[0,:]
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

# to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()


roi_number = brain_voxel_list[0]
brain_voxel_list = brain_voxel_list[1:]

x_dim, y_dim, z_dim =mask.shape

brain_voxel_counter = 0
for i in range(x_dim):
    for j in range(y_dim):
        for k in range(z_dim):
            if mask[i,j,k] == 1:
                brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                brain_voxel_counter = brain_voxel_counter + 1




# np.max(mask)



## Checking if adding a header and saving the matrix as brain image doesnot alter its contents

In [68]:
np.diag(brain[45,:,:])


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.12681171,  0.37199557,  0.        ,  0.70967722,  0.89365005,
        0.3016859 ,  0.68334228,  0.86271942,  0.4511511 ,  0.8674804 ,
        0.97397631,  0.442323  ,  0.35323575,  0.35975546,  0.54950088,
        0.7498036 ,  0.93263155,  0.77791846,  0.        ,  0.        ,
        0.89896637,  0.74759054,  0.50572908,  0.76082379,  0.5428375 ,
        0.72145051,  0.93261176,  0.73828477,  0.30564788,  0.39948294,
        0.54905254,  0.79104036,  0.49730939,  0.5246681 ,  0.51921791,
        0.05631606,  0.03869139,  0.20976715,  0.96198726,  0.37815624,
        0.86623013,  0.33423769,  0.10890429,  0.12835464,  0.98279679,
        0.62302142,  0.52715665,  0.87349391,  0.56189305,  0.20

In [69]:
brain_with_header = nib.Nifti1Image(brain, affine=brain_data.affine,header = brain_data.header)

In [71]:
brain = brain_with_header.get_data()
np.diag(brain[45,:,:])

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.12681171,  0.37199557,  0.        ,  0.70967722,  0.89365005,
        0.3016859 ,  0.68334228,  0.86271942,  0.4511511 ,  0.8674804 ,
        0.97397631,  0.442323  ,  0.35323575,  0.35975546,  0.54950088,
        0.7498036 ,  0.93263155,  0.77791846,  0.        ,  0.        ,
        0.89896637,  0.74759054,  0.50572908,  0.76082379,  0.5428375 ,
        0.72145051,  0.93261176,  0.73828477,  0.30564788,  0.39948294,
        0.54905254,  0.79104036,  0.49730939,  0.5246681 ,  0.51921791,
        0.05631606,  0.03869139,  0.20976715,  0.96198726,  0.37815624,
        0.86623013,  0.33423769,  0.10890429,  0.12835464,  0.98279679,
        0.62302142,  0.52715665,  0.87349391,  0.56189305,  0.20

## Check Successful.

## ------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Checking the brain header again

brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()

brain_with_header = nib.Nifti1Image(mask, affine=brain_data.affine,header = brain_data.header)



print (brain_data.header)


## Execute the above created node to create the p-value brain images

In [74]:
# create a pool of workers and distribute the p_value_brain_creater among them 
# to create multiple p-value files in parallel
from multiprocessing import Pool
# os.chdir("../result/p_value_files/") to save the results at desired position. Coz node saves at a temporary location.
            
pool = Pool(8)
pool_inputs = [] #np.arange(number_of_ROIs)

for roi in range(num_ROIs):
    pool_inputs.append(X[roi,:])
    
brain_file_names = pool.map(p_value_brain_creater, pool_inputs)

All p-values successfully fitted ROI: 1.0
Saved!
All p-values successfully fitted ROI: 10.0
Saved!
All p-values successfully fitted ROI: 19.0
Saved!
All p-values successfully fitted ROI: 28.0
All p-values successfully fitted ROI: 37.0
All p-values successfully fitted ROI: 46.0
Saved!
Saved!
All p-values successfully fitted ROI: 55.0
Saved!
Saved!
All p-values successfully fitted ROI: 64.0
Saved!
All p-values successfully fitted ROI: 2.0
Saved!
All p-values successfully fitted ROI: 11.0
Saved!
All p-values successfully fitted ROI: 20.0
Saved!
All p-values successfully fitted ROI: 29.0
All p-values successfully fitted ROI: 38.0
Saved!
Saved!
All p-values successfully fitted ROI: 56.0
All p-values successfully fitted ROI: 47.0
Saved!
Saved!
All p-values successfully fitted ROI: 65.0
Saved!
All p-values successfully fitted ROI: 3.0
Saved!
All p-values successfully fitted ROI: 12.0
Saved!
All p-values successfully fitted ROI: 21.0
Saved!
All p-values successfully fitted ROI: 39.0
Saved!
All

Saved!
All p-values successfully fitted ROI: 174.0
Saved!
All p-values successfully fitted ROI: 192.0
Saved!
All p-values successfully fitted ROI: 183.0
Saved!
All p-values successfully fitted ROI: 201.0
Saved!
All p-values successfully fitted ROI: 148.0
Saved!
All p-values successfully fitted ROI: 157.0
Saved!
All p-values successfully fitted ROI: 166.0
Saved!
All p-values successfully fitted ROI: 210.0
Saved!
All p-values successfully fitted ROI: 175.0
Saved!
All p-values successfully fitted ROI: 184.0
All p-values successfully fitted ROI: 193.0
Saved!
Saved!
All p-values successfully fitted ROI: 202.0
Saved!
All p-values successfully fitted ROI: 149.0
Saved!
All p-values successfully fitted ROI: 158.0
Saved!
All p-values successfully fitted ROI: 211.0
All p-values successfully fitted ROI: 167.0
Saved!
Saved!
All p-values successfully fitted ROI: 176.0
Saved!
All p-values successfully fitted ROI: 185.0
All p-values successfully fitted ROI: 194.0
Saved!
Saved!
All p-values successfull

In [31]:
brain_voxel_list = pool_inputs[0]

In [47]:
p_val_data = nib.load(brain_file_names[0])
p_val = p_val_data.get_data()


In [50]:
rn = np.random.rand(10)
rn

array([ 0.24696897,  0.36937996,  0.00960354,  0.94996886,  0.72696877,
        0.37071412,  0.03425176,  0.76267062,  0.63455416,  0.4262049 ])

# Do FDR Corretion and store the result in a single file

# TODO 
Final work: 
* Create a memory Manager to create a Shared 4D array of size [x_dim,y_dim,z_dim,ROIs]  
* Create pool of 8 workers
* Every worker calculates the FDR corrected value
* Reshapes it and stores its result at the ROI position in the 4D array
* Then store this 4D array along with some functional file's header 

In [15]:
import multiprocessing.managers
from multiprocessing import Pool
# os.chdir("../result/p_value_files/")


class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)


# Read mask files:
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

# read the file on which you want to write the p-values
brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()

# Edit header

(brain_data.header).set_data_shape([91,109,91,num_ROIs])



In [62]:
def do_fdr(fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask,brain_voxel_list):
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()
    
    # to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
    brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
    brain_data = nib.load(brain_file)
    
    
    # Two empty brain to store the fdr corrected values and mask
    brain = brain_data.get_data()
    
    brain_corrected_mask = np.zeros((brain.shape))

    
#     global fdrcorrected_brain_4Dtensor
#     global fdrcorrected_brain_4Dtensor_mask
    
    roi_number = int(brain_voxel_list[0]) - 1
    brain_voxel_list = brain_voxel_list[1:]
    
    # FDR Correction: http://www.statsmodels.org/devel/generated/statsmodels.sandbox.stats.multicomp.fdrcorrection0.html#statsmodels.sandbox.stats.multicomp.fdrcorrection0
    
    alpha = 0.05
    from statsmodels.sandbox.stats.multicomp import fdrcorrection0

    brain_voxel_list_mask, brain_voxel_list  = fdrcorrection0(brain_voxel_list,alpha)
    
    
    
    
    
    x_dim, y_dim, z_dim = mask.shape

    brain_voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                    if brain_voxel_list_mask[brain_voxel_counter] == True:
                        brain_corrected_mask[i,j,k] = 1
                    brain_voxel_counter = brain_voxel_counter + 1
                    

    
    
    if brain_voxel_counter == len(brain_voxel_list):
        print("All p-values successfully fitted ROI:",roi_number)
#         filename = 'roi_'+ str(int(roi_number)) + '_p_value_file.nii.gz'
        
        
#         brain_with_header = nib.Nifti1Image(brain, affine=brain_data.affine,header = brain_data.header)
#         nib.save(brain_with_header,filename)
        
    
        print('Brain shape ',brain.shape)
        fdrcorrected_brain_4Dtensor[:,:,:,roi_number] = brain
        fdrcorrected_brain_4Dtensor_mask[:,:,:,roi_number] = brain_corrected_mask

        print('Stored in shared arrays!')

#         p_value_file = opj(os.getcwd(),filename) # path of file
        
        return 1
    else:
        print("p-values fitting failed")
        return 0

In [63]:
X[:,0] # appended the ROI number to the ROI-voxels correlation vector

array([   1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,    9.,
         10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,   18.,
         19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,   27.,
         28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,   36.,
         37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,   45.,
         46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,   54.,
         55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,   63.,
         64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,   72.,
         73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,   81.,
         82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,   90.,
         91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,   99.,
        100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,  108.,
        109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,  117.,
        118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,  126.,
        127.,  128.,

In [64]:
%%time
# Create pool of 8 workers

class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)
from functools import partial
            
pool_inputs = [] #np.arange(number_of_ROIs)

for roi in range(num_ROIs):
    pool_inputs.append(X[roi,:])
    

# print (brain_data.header)


m = MyManager()
m.start()
fdrcorrected_brain_4Dtensor = m.np_zeros((brain_data.header.get_data_shape()))
fdrcorrected_brain_4Dtensor_mask = m.np_zeros((brain_data.header.get_data_shape()))

func = partial(do_fdr, fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask)

pool = Pool(8)

data_outputs = pool.map(func, pool_inputs)



# Saving tthe fdr corrected brains: 
brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains.nii.gz')

brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor_mask, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_mask.nii.gz')


All p-values successfully fitted ROI: 9
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 0
All p-values successfully fitted ROI: 18
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 36
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 54
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 63
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 45
Stored in shared arrays!
Stored in shared arrays!
Stored in shared arrays!
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 27
Stored in shared arrays!
Brain shape  (91, 109, 91)
Stored in shared arrays!
Stored in shared arrays!
Stored in shared arrays!
All p-values successfully fitted ROI: 19
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 37
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 1
All p-values successfully fitted ROI: 55
Brain shape  (91, 109, 91)
All p-values s

Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 92
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 83
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 110
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 101
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 128
Stored in shared arrays!
Brain shape  (91, 109, 91)
Stored in shared arrays!
Stored in shared arrays!
All p-values successfully fitted ROI: 119
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 137
Brain shape  (91, 109, 91)
Stored in shared arrays!
Stored in shared arrays!
All p-values successfully fitted ROI: 75
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 93
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 84
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 

All p-values successfully fitted ROI: 148
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 157
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 166
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 175
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 193
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 184
Brain shape  (91, 109, 91)
All p-values successfully fitted ROI: 202
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 211
Brain shape  (91, 109, 91)
Stored in shared arrays!
Stored in shared arrays!
All p-values successfully fitted ROI: 149
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 158
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 167
Brain shape  (91, 

Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 259
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 268
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 233
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 250
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 241
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 224
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 260
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 269
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 251
Brain shape  (91, 109, 91)
Stored in shared arrays!
All p-values successfully fitted ROI: 242
Brain shape  (91, 109, 91)
Stored in shared arrays!


AttributeError: 'ArrayProxy' object has no attribute 'shape'

In [68]:
# CONVERTING THE ArrayProxy to numpy array
fdrcorrected_brain_4Dtensor = np.asarray(fdrcorrected_brain_4Dtensor)
fdrcorrected_brain_4Dtensor_mask = np.asarray(fdrcorrected_brain_4Dtensor_mask)


# Saving tthe fdr corrected brains: 
brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains.nii.gz')

brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor_mask, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_mask.nii.gz')


# Things Learned
* Parallel Execution of a piece of code over multiple cores
* Sending shared array to multiple cores
* Reading a Big numpy array in chunks
* Modifying the dimensions of data in nifti header

## Parallel Execution of a piece of code over multiple cores 
* If you have a for loop and you want to distribute it over multiple processors

In [60]:
brain.shape

(91, 109, 91, 176)

In [38]:
hdr = brain_data.header
hdr.set_data_shape([91,109,91,274])
print(brain_data.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 48
dim             : [  4  91 109  91 274   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  2.  2.  2.  2.  1.  1.  1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 90
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : mni
sform_code      : mni
quatern_b       : -0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [ -2.  -0.  -0.  90.]
srow

In [ ]:
/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz

In [100]:
np.diag(fdrcorrected_brain_4Dtensor[:,:,40,40])

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.98325729,  0.97170764,  0.98390532,  0.88090324,
        0.82788301,  0.82788301,  0.98435867,  0.97825104,  0.9770233 ,
        0.96220672,  0.94705951,  0.99889594,  0.96339422,  0.8620162 ,
        0.83969271,  0.86213827,  0.93656141,  0.90644336,  0.86255974,
        0.83969271,  0.88167185,  0.9024958 ,  0.98325729,  0.99367869,
        0.96400422,  0.998357  ,  0.99689084,  0.99857956,  0.99388719,
        0.99223697,  0.92462164,  0.92737627,  0.92232633,  0.98713285,
        0.95337296,  0.87492484,  0.86971307,  0.96901363,  0.96604025,
        0.98714352,  0.9966656 ,  0.97527486,  0.99554652,  0.87757188,
        0.83969271,  0.91037184,  0.96860635,  0.95767516,  0.90

In [54]:
alpha = 0.1
from statsmodels.sandbox.stats.multicomp import fdrcorrection0

fdrcorrection0(rn,alpha)

(array([False, False,  True, False, False, False, False, False, False, False], dtype=bool),
 array([ 0.7103415 ,  0.7103415 ,  0.09603539,  0.94996886,  0.8474118 ,
         0.7103415 ,  0.17125881,  0.8474118 ,  0.8474118 ,  0.7103415 ]))

In [32]:
roi_number = brain_voxel_list[0]
brain_voxel_list = brain_voxel_list[1:]

In [34]:
brain_voxel_list

array([ 0.00328066,  0.01261732,  0.05239893, ...,  0.36491533,
        0.97971208,  0.48926658])

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3  91 109  91   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  2.  2.  2.  1.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'FSL5.0'
aux_file        : b''
qform_code      : mni
sform_code      : mni
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [ -2.   0.   0.  90.]
s

In [24]:
np.min(X[3,:]), np.max(X[1,1:])

(3.6899537504319097e-06, 0.99999771815771044)

In [ ]:
# now use FDR of fsl to read all the file names and 



brain_with_header = nib.Nifti1Image(brain, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,func_brain_path)


In [56]:
D4 = np.random.rand(5,5,5,5)

a = D4[:,:,:,1.0]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
brain.shape

In [ ]:
91*109*91

In [ ]:
num_voxels_in_ROI.max()

In [ ]:
!pwd

In [84]:
# Results looked awful.. Showed up nothing
brain_voxel_list = X[0,:]

alpha = 0.05
from statsmodels.sandbox.stats.multicomp import fdrcorrection0

brain_voxel_list_mask, brain_voxel_list  = fdrcorrection0(brain_voxel_list,alpha)

In [95]:
brain_voxel_list_mask
# brain_voxel_list_mask[brain_voxel_list_mask == True]

array([False, False, False, ..., False, False, False], dtype=bool)

In [86]:
brain_voxel_list

array([ 1.       ,  0.9999827,  0.9999827, ...,  0.9999827,  0.9999827,
        0.9999827])

In [76]:
np.min(X[1:,:])

2.1609759252534177e-08

In [ ]:
I Think there is some problem with p values-- they look a bit big. Come back and check again

In [90]:
Y = np.load('/home1/varunk/significance_matrix.npy', mmap_mode='r')

In [93]:
 Y[1,:]

memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [ ]:
/home1/varunk/significance_matrix_2.npy